### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/fedesoriano/heart-failure-prediction
2. Соберем пайплайн.
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [22]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Загрузим данные

In [24]:
df = pd.read_csv("./heart.csv.xls")
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [25]:
# df.info()
# df.isna().sum()

Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['HeartDisease'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [27]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

Соберем кусок, ответственный за feature engineering

In [28]:
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [29]:
cat_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
cont_columns = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
final_transformers = list()


for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))

for cont_col in cont_columns:
    cont_transformer = Pipeline([
                ('imputer', TextImputer(cont_col, '')),
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))


feats = FeatureUnion(final_transformers)
# feature_processing = Pipeline([('feats', feats)])

Добавим простейший классификатор

In [30]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 98.1 ms, sys: 23.7 ms, total: 122 ms
Wall time: 69.6 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('RestingECG',
                                                 Pipeline(steps=[('selector',
    

Посмотрим, как выглядит наш pipeline

In [31]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Sex',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='Sex')),
                                                  ('ohe',
                                                   OHEEncoder(key='Sex'))])),
                                 ('ChestPainType',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='ChestPainType')),
                                                  ('ohe',
                                                   OHEEncoder(key='ChestPainType'))])),
                                 ('RestingECG',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='RestingECG')),
                                                  ('ohe',
                                                   

Сохраним модель (пайплайн)

In [32]:
# dmodel = {
#     'model': pipeline,
#     'ver': '1.1'
# } так можно сохранять версионность

In [33]:
# with open("logreg_pipeline.dill", "wb") as f:
#     dill.dump(dmodel, f)

In [34]:
with open("./models/logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)